# Don't eat dog shit

Here's a little game you can play with a friend. Say you have $n-1$ pieces of cage and one dog shit.

In [1]:
items = ["💩", "🍰", "🍰", "🍰", "🍰"]
n = len(items)

You will now take turns picking up, from the end of the list, one, two, or three items and eat them. Then your friend will take one, two, or three items. Then you again. And you will continue like that until there is nothing left.

You want to eat cake, but obviously not shit, so you want to pick the number of items you take such that you do not end up having to take the last piece.

It is unlikely that you have any friends who would want to play this game with you, so we will program a player as a computer program instead. We set up the game using this code:

In [2]:
shit = ["💩"]
cake = ["🍰"]
from random import randint

def get_initial_state(min_n = 10, max_n = 15):
    """
    Construct an initial state of the game,
    represented as a list.
    """
    num_cakes = randint(min_n - 1, max_n - 1)
    return shit + num_cakes * cake

We can test the function by calling it.

In [3]:
get_initial_state()

['💩', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰', '🍰']

Now, for implementing the steps in the game, we write a function for your moves and one for the computer's moves. For the latter, we start out with picking a random number of items eat.

When we play the game, we will print the state so you can. The computer cannot see what we print in its choice function so we have to provide information about the state to it. Otherwise, it will not have the same information that you have, which would be unfair. If we tell it what the number of items left are, before it has to choose a number of items to eat, then it will have the same information as you do. To make the two move functions similar enough that we do not have to worry about which we are calling later, we provide the same information to your move function as well.

In [4]:
def your_move(n):
    """
    Prompt you for your choice.
    """
    while True:
        try:
            choice = input("How many items do you take? ")
            choice = int(choice)
            if choice not in [1,2,3]:
                print("You have to choose one of 1, 2, or 3.")
                continue
            return choice
        except:
            print("You did not input an integer.")

In [5]:
from random import randint
def computer_move(n):
    """
    Make the computer choose a number of items to take.
    """
    return randint(1,3)

OK, with this simple strategy, it doesn't actually use the state information, but we will come up with a smarter strategy later.

The entire game can now be implemented as a look that runs until the last pieces is taken.

In [6]:
def run_game(min_n = 10, max_n = 15):
    """
    Set up the game and run it until the last item is taken.
    """
    players = [your_move, computer_move]
    player_names = ["You", "Computer"]
    turn = 0
    state = get_initial_state(min_n, max_n)
    n = len(state)
    
    while n > 0:
        print("State: {}".format("".join(state)))
        print("Turn: {}".format(player_names[turn % 2]))
        choice = players[turn % 2](n)
        while choice > n: # don't pick too high a number
            print("There are only {} items left!".format(n))
            choice = players[turn % 2](n)
        print("Taking {} items.\n".format(choice))
        state = state[:-choice]
        n = len(state)
        turn += 1
        
    # turn is incremented in the last loop, so turn-1 % 2
    # is the player that took the last item.
    if (turn - 1) % 2 == 0:
        print("You loose! 🤮")
    else:
        print("You win 😓")

In [7]:
run_game()

State: 💩🍰🍰🍰🍰🍰🍰🍰🍰🍰🍰🍰🍰🍰
Turn: You
How many items do you take? 3
Taking 3 items.

State: 💩🍰🍰🍰🍰🍰🍰🍰🍰🍰🍰
Turn: Computer
Taking 2 items.

State: 💩🍰🍰🍰🍰🍰🍰🍰🍰
Turn: You
How many items do you take? 2
Taking 2 items.

State: 💩🍰🍰🍰🍰🍰🍰
Turn: Computer
Taking 3 items.

State: 💩🍰🍰🍰
Turn: You
How many items do you take? 3
Taking 3 items.

State: 💩
Turn: Computer
There are only 1 items left!
There are only 1 items left!
There are only 1 items left!
There are only 1 items left!
Taking 1 items.

You win 😓


## Implementing a clever AI

The computer strategy is to pick a random number. This might be good enough to beat you on rare occasions, but we can come up with a smarter strategy for it. Maybe, something like this:

In [8]:
def computer_move(n):
    return max(1, (n-1) % 4)

Try to beat it now.

In [9]:
run_game()

State: 💩🍰🍰🍰🍰🍰🍰🍰🍰🍰
Turn: You
How many items do you take? 3
Taking 3 items.

State: 💩🍰🍰🍰🍰🍰🍰
Turn: Computer
Taking 2 items.

State: 💩🍰🍰🍰🍰
Turn: You
How many items do you take? 2
Taking 2 items.

State: 💩🍰🍰
Turn: Computer
Taking 2 items.

State: 💩
Turn: You
How many items do you take? 1
Taking 1 items.

You loose! 🤮


Maybe you consider this too simple to be real artificial intelligence, but as a computer program, it will be hard to beat. If you are unlucky in the choice of initial state, or if you make a single mistake, it will beat you.

Of course, the formula `max(1, (n-1) % 4)` wasn't chosen arbitrarily. If $n$ is chosen randomly, this strategy will win three times out of four against a player that never makes mistakes. To see this, and as a general approach to coming up with moves in a game, we can tabulate the different states, moves, and whether we have won or lost.

|    State | Your move | Next state | Win or loose? |
|----------|-----------|------------|---------------|
|💩        | 1         |            | 🤮            |
|💩🍰      | 1         | 💩         | 😓            |
|💩🍰      | 2         |            | 🤮            |
|💩🍰🍰    | 1         | 💩🍰      | ?            |
|💩🍰🍰    | 2         | 💩        | 😓            |
|💩🍰🍰    | 3         |           | 🤮            |
|💩🍰🍰🍰  | 1         | 💩🍰🍰    | ?            |
|💩🍰🍰🍰  | 2         | 💩🍰      | ?            |
|💩🍰🍰🍰  | 3         | 💩        | 😓           |
|💩🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰    | ?            |
|💩🍰🍰🍰🍰  | 2         | 💩🍰🍰      | ?            |
|💩🍰🍰🍰🍰  | 3         | 💩🍰        | ?           |
|💩🍰🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰🍰    | ?            |
|💩🍰🍰🍰🍰🍰  | 2         | 💩🍰🍰🍰      | ?            |
|💩🍰🍰🍰🍰🍰  | 3         | 💩🍰🍰        | ?           |
| ...           | ...       | ...           | ...       |

and the table continues for longer and longer states.

In this table, there are some states and moves where we can immediately see if we win or loose. If we only have 💩 left when we have to make a move, we will loose. If we can leave the state after our moves as only 💩, we win. If we had a choice and *could* have avoided eating 💩, but did so anyway, we still loose. Those are cases we can, and should, avoid. We can remove those choices and we have this table:

|    State | Your move | Next state | Win or loose? |
|----------|-----------|------------|---------------|
|💩        | 1         |            | 🤮            |
|💩🍰      | 1         | 💩         | 😓            |
|💩🍰🍰    | 1         | 💩🍰      | ?            |
|💩🍰🍰    | 2         | 💩        | 😓            |
|💩🍰🍰🍰  | 1         | 💩🍰🍰    | ?            |
|💩🍰🍰🍰  | 2         | 💩🍰      | ?            |
|💩🍰🍰🍰  | 3         | 💩        | 😓           |
|💩🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰    | ?            |
|💩🍰🍰🍰🍰  | 2         | 💩🍰🍰      | ?            |
|💩🍰🍰🍰🍰  | 3         | 💩🍰        | ?           |
|💩🍰🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰🍰    | ?            |
|💩🍰🍰🍰🍰🍰  | 2         | 💩🍰🍰🍰      | ?            |
|💩🍰🍰🍰🍰🍰  | 3         | 💩🍰🍰        | ?           |
| ...           | ...       | ...           | ...       |

We cannot remove the loss in state 💩, though. This is the state we want to avoid, so we have to come up with a strategy for that.

In some states we now have a move that we know will lead us to a win because the new state forces the computer to eat shit. If we are in state 💩🍰 and pick one item, we win. If we are in state 💩🍰🍰 and pick 2, we win. If we are in state 💩🍰🍰🍰, we win. We can ignore the other choices we might have made in those states, because we shouldn't make them. We already know a winning move from those states.

|    State | Your move | Next state | Win or loose? |
|----------|-----------|------------|---------------|
|💩        | 1         |            | 🤮            |
|💩🍰      | 1         | 💩         | 😓            |
|💩🍰🍰    | 2         | 💩        | 😓            |
|💩🍰🍰🍰  | 3         | 💩        | 😓           |
|💩🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰    | ?            |
|💩🍰🍰🍰🍰  | 2         | 💩🍰🍰      | ?            |
|💩🍰🍰🍰🍰  | 3         | 💩🍰        | ?           |
|💩🍰🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰🍰    | ?            |
|💩🍰🍰🍰🍰🍰  | 2         | 💩🍰🍰🍰      | ?            |
|💩🍰🍰🍰🍰🍰  | 3         | 💩🍰🍰        | ?           |
| ...           | ...       | ...           | ...       |


The strategy we want should always pick a winning move when it can, but we only know three winning moves (and one guaranteed loss) so far. How do we move from here?

We can build up the table by considering the states from shortest to longest. If we look at a state and know the outcome for all shorter states, we can make a choice based on those. Of course, we cannot know with absolute certainty what will happen in the game for the shorter states--that depends on the choices the other player makes--but if we assume that he is as smart as we are, then we will have to assume that if he ends up in a state that we know we could win from, putting him in that state means that we will loose.

Let us consider the shortest state where we do not know the outcome, 💩🍰🍰🍰🍰. From this state, we will end up in one of these: 💩🍰🍰🍰, 💩🍰🍰, or 💩🍰. We know that if *we* started in any of these states, we would win, so we must assume that if our adversary starts in one of these states *he* will win, i.e. we will loose. Since all out choices lead to loosing, the out come for this entire state is a loss, and we can update the table accordingly.

|    State | Your move | Next state | Win or loose? |
|----------|-----------|------------|---------------|
|💩        | 1         |            | 🤮            |
|💩🍰      | 1         | 💩         | 😓            |
|💩🍰🍰    | 2         | 💩        | 😓            |
|💩🍰🍰🍰  | 3         | 💩        | 😓           |
|💩🍰🍰🍰🍰  | ?         | 💩🍰🍰🍰    | 🤮            |
|💩🍰🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰🍰    | ?            |
|💩🍰🍰🍰🍰🍰  | 2         | 💩🍰🍰🍰      | ?            |
|💩🍰🍰🍰🍰🍰  | 3         | 💩🍰🍰        | ?           |
| ...           | ...       | ...           | ...       |

Now, the shortest state is this: 💩🍰🍰🍰🍰🍰. Our possible choices leads us to one of these states, where we know the outcome: 💩🍰🍰🍰🍰 (🤮), 💩🍰🍰🍰 (😓), 💩🍰🍰 (😓). Here, the choice matters, but we can write down the outcome dependent on the choice.

|    State | Your move | Next state | Win or loose? |
|----------|-----------|------------|---------------|
|💩        | 1         |            | 🤮            |
|💩🍰      | 1         | 💩         | 😓            |
|💩🍰🍰    | 2         | 💩        | 😓            |
|💩🍰🍰🍰  | 3         | 💩        | 😓           |
|💩🍰🍰🍰🍰  | ?         | 💩🍰🍰🍰    | 🤮            |
|💩🍰🍰🍰🍰🍰  | 1         | 💩🍰🍰🍰🍰    | 🤮            |
|💩🍰🍰🍰🍰🍰  | 2         | 💩🍰🍰🍰      | 😓            |
|💩🍰🍰🍰🍰🍰  | 3         | 💩🍰🍰        | 😓           |
| ...           | ...       | ...           | ...       |

The states we mark as loosing here aren't guaranteed losses. If the adversary makes a mistake we are back in the game (and if we never make a mistake, because we have a winning strategy, then we will win after the first mistake he makes). If we are in a state we have marked as winning, we can always make a transition that moves us to another winning state, which is the same as saying that we move to a state where the adversary is in a loosing state and have to make a choice that leaves *us* in a winning state.

**Exercise:** Prove that for all $n$ where $(n-1)\mod 4 > 0$ we are in a winning state.

If we are in a state $n$ where $(n-1)\mod 4 = 0$, then we are in a loosing state, and it doesn't matter what we do. We can only hope that, at some later point, the adversary will make a mistake.

**Exercise:** Prove from this, that the strategy $\max(1,(n-1)\mod 4)$ is the best we can do.